# Results 
- Plot training/test telemetry values, predictions, smoothed errors, and predicted and actual anomalies
- A specified results file from the `results` dir is used to highlight anomalous regions and read in data from the `data` dir

In [1]:
import sys
import numpy as np
import os
sys.path.insert(0, '..')
import telemanom.helpers as helpers
from telemanom.plotting import Plotter
import pandas as pd
import plotly as py
import matplotlib.pyplot as plt
from plotly.offline import download_plotlyjs, init_notebook_mode
import cufflinks as cf
import glob
cf.go_offline()
init_notebook_mode(connected=True)

%load_ext autoreload
%autoreload 2

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

## Examine labeled_anomalies.csv

In [3]:
with pd.option_context('display.max_colwidth', 400):
    la_file = "../labeled_anomalies.csv"
    values = pd.read_csv(la_file)
    values = values[values["spacecraft"] == "SMAP"]
    print(values.columns)
    print(values.head())
    print()
    fn = "P-1"
    indices = values[values['chan_id'] == fn]["anomaly_sequences"].values[0]
    print(indices)
    print(type(indices))
    indices = indices.replace(']', '').replace('[', '').split(', ')
    print(indices)
    print(type(indices))
    val = np.load(f"/home/alexey/School/Research/submodules/telemanom/data/test/{fn}.npy")
    labels = np.zeros(val.shape)
    indices = [int(i) for i in indices]
    for i in range(0, len(indices), 2):
        print(f"[{indices[i]}:{indices[i+1]}, :]")
        labels[indices[i]:indices[i+1], :] = 1

Index(['chan_id', 'spacecraft', 'anomaly_sequences', 'class', 'num_values'], dtype='object')
  chan_id spacecraft                           anomaly_sequences  \
0     P-1       SMAP  [[2149, 2349], [4536, 4844], [3539, 3779]]   
1     S-1       SMAP                              [[5300, 5747]]   
2     E-1       SMAP                [[5000, 5030], [5610, 6086]]   
3     E-2       SMAP                              [[5598, 6995]]   
4     E-3       SMAP                              [[5094, 8306]]   

                                  class  num_values  
0  [contextual, contextual, contextual]        8505  
1                               [point]        7331  
2              [contextual, contextual]        8516  
3                               [point]        8532  
4                               [point]        8307  

[[2149, 2349], [4536, 4844], [3539, 3779]]
<class 'str'>
['2149', '2349', '4536', '4844', '3539', '3779']
<class 'list'>
[2149:2349, :]
[4536:4844, :]
[3539:3779, :]


In [4]:
fn = "P-1"
fn_labels = np.load(f"/home/alexey/School/Research/submodules/TranAD/processed/SMAP/{fn}_labels.npy")
print(fn_labels[2148:2150, :])

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1.]]


## Print Precision and Recall

In [5]:
run_ids = ["2018-05-19_15.00.10", "yes_avg", "no_avg"]

for run_id in run_ids:
    result_df = pd.read_csv(os.path.join('..', 'results', '{}.csv'.format(run_id)))

    print(run_id)

    sum_df = result_df.loc[:55,["true_positives", "false_negatives", "false_positives"]]
    sum_df = sum_df.sum()

    print("  SMAP:")
    precision = float(sum_df['true_positives']) / float(sum_df['true_positives'] + sum_df['false_positives'])
    recall = float(sum_df['true_positives']) / float(sum_df['true_positives'] + sum_df['false_negatives'])
    f = lambda beta: (1. + beta**2) * ( (precision*recall) / ( ((beta**2)*precision) + recall )) 
    print('    Precision: {0:.2f}'.format(precision))
    print('    Recall: {0:.2f}'.format(recall))
    print('    F0.5: {0:.2f}'.format(f(0.5)))
    print('    F1.0: {0:.2f}'.format(f(1.0)))
    
    print()

2018-05-19_15.00.10
  SMAP:
    Precision: 0.84
    Recall: 0.90
    F0.5: 0.85
    F1.0: 0.87

yes_avg
  SMAP:
    Precision: 0.84
    Recall: 0.90
    F0.5: 0.85
    F1.0: 0.87

no_avg
  SMAP:
    Precision: 0.92
    Recall: 0.87
    F0.5: 0.91
    F1.0: 0.90



## Print SMAP and MSL streams

In [6]:
run_id = "2018-05-19_15.00.10"
result_df = pd.read_csv(os.path.join('..', 'results', '{}.csv'.format(run_id)))
SMAP_IDs = result_df[result_df["spacecraft"] == "SMAP"]\
    .loc[:, ["chan_id"]]
print(f"SMAP ({len(SMAP_IDs)}):")
print(SMAP_IDs)
print()
MSL_IDs = result_df[result_df["spacecraft"] == "MSL"]\
    .loc[:, ["chan_id"]]
print(f"MSL ({len(MSL_IDs)}):")
print(MSL_IDs)

SMAP (55):
   chan_id
0      P-1
1      S-1
2      E-1
3      E-2
4      E-3
5      E-4
6      E-5
7      E-6
8      E-7
9      E-8
10     E-9
11    E-10
12    E-11
13    E-12
14    E-13
15     A-1
16     D-1
17     P-2
18     P-3
19     D-2
20     D-3
21     D-4
22     A-2
23     A-3
24     A-4
25     G-1
26     G-2
27     D-5
28     D-6
29     D-7
30     F-1
31     P-4
32     G-3
33     T-1
34     T-2
35     D-8
36     D-9
37     F-2
38     G-4
39     T-3
40    D-11
41    D-12
42     B-1
43     G-6
44     G-7
45     P-7
46     R-1
47     A-5
48     A-6
49     A-7
50    D-13
51     P-2
52     A-8
53     A-9
54     F-3

MSL (27):
   chan_id
55     M-6
56     M-1
57     M-2
58     S-2
59    P-10
60     T-4
61     T-5
62     F-7
63     M-3
64     M-4
65     M-5
66    P-15
67     C-1
68     C-2
69    T-12
70    T-13
71     F-4
72     F-5
73    D-14
74     T-9
75    P-14
76     T-8
77    P-11
78    D-15
79    D-16
80     M-7
81     F-8


In [7]:
chan_id = "S-1"
val = np.load(f"/home/alexey/School/Research/submodules/telemanom/data/test/{chan_id}.npy")
print(val.shape)


(7331, 25)


## Select set of results to visualize

In [10]:
#run_ids = ["2018-05-19_15.00.10", "yes_avg", "no_avg"]
#run_ids = ["yes_avg", "no_avg"]
run_ids = ["2018-05-19_15.00.10"]

In [11]:
for k in range(len(run_ids)):
    with open('../data/{}/params.log'.format(run_ids[k]), 'r') as f:
        print(run_ids[k])
        for i,row in enumerate(f.readlines()):
            if len(row) < 2:
                break
            if i > 0:
                print(' '.join(row.split(' ')[3:]).replace('\n',''))

2018-05-19_15.00.10
----------------
batch_size: 70
dropout: 0.3
epochs: 35
error_buffer: 100
l_s: 250
layers: [80, 80]
loss_metric: mse
lstm_batch_size: 64
min_delta: 0.0003
n_predictions: 10
optimizer: adam
p: 0.13
patience: 10
predict: False
smoothing_perc: 0.05
train: False
validation_split: 0.2
window_size: 30
----------------


## Summary of results

In [12]:
for k in range(len(run_ids)):
    print(run_ids[k])
    plotter = Plotter(run_ids[k])
    plotter.all_result_summary()
    print()

2018-05-19_15.00.10
True Positives: 87
False Positives: 13
False Negatives: 18

Precision: 0.87
Recall: 0.83



## Interactive inline Plotly charts for viewing `y_test`, `y_hat`, and `smoothed errors (e_s)`
- **Blue** highlighted regions indicate anomalous sequences detected by the system
- If available, **Red** highlighted regions indicate true anomalous regions
- Can also optionally plot training data by setting `plot_train=True`

In [24]:
for i in range(len(run_ids)):
    print(run_ids[i])
    plotter = Plotter(run_ids[i])
    plotter.plot_channel('A-9', plot_errors=True, plot_train=True, plot_test=True)
    plotter.channel_result_summary('A-9')
    print()

2018-05-19_15.00.10
Train shape: (762, 25)
Test shape: (8434, 25)
Spacecraft: SMAP
Channel: A-9
Normalized prediction error: 0.47
Anomaly class(es): [contextual]
------------------
True Positives: 0
False Positives: 0
False Negatives: 1
------------------
Predicted anomaly scores: []
Number of values: 8434


Spacecraft: SMAP
Channel: A-9
Normalized prediction error: 0.47
Anomaly class(es): [contextual]
------------------
True Positives: 0
False Positives: 0
False Negatives: 1
------------------
Predicted anomaly scores: []
Number of values: 8434



In [ ]:
#plotter.plot_all(plot_errors=True, plot_train=False)